In [30]:
import cv2
import requests
import numpy as np
import os
import time
from nba_api.stats.endpoints import leaguegamefinder

In [4]:
def get_season_games(season_year='2019-20'):
    gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season_year)
    games_df = gamefinder.get_data_frames()[0]
    return games_df['GAME_ID'].unique()

game_ids = get_season_games('2019-20')
game_ids

array(['0041900406', '0041900405', '0041900404', ..., '0011900002',
       '0011900004', '0011900001'], dtype=object)

In [24]:
subsetGMs=game_ids[350:360:2]
print(subsetGMs)

['2021900513' '2021900509' '0021900883' '0021900879' '0021900884']


In [35]:
headers = {
    'Host': 'stats.nba.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'x-nba-stats-origin': 'stats',
    'x-nba-stats-token': 'true',
    'Connection': 'keep-alive',
    'Referer': 'https://stats.nba.com/',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache'
}

In [49]:
successful_video_eventssuccessful_video_events=[]
for gid in subsetGMs:
    for i in range(200):
        time.sleep(.2)
        if (i+1)%50 == 0:
            print(str((i+1)/2)+'% of', gid ,'complete')
        video_count=0
        event_id = i
        game_id = gid
        url = 'https://stats.nba.com/stats/videoeventsasset?GameEventID={}&GameID={}'.format(
            event_id, game_id)
        try:
            r = requests.get(url, headers=headers)
            json = r.json()
            video_urls = json['resultSets']['Meta'].get('videoUrls', [])
            playlist = json['resultSets'].get('playlist', [])
            
            if video_urls and playlist:  # Ensure there are items before accessing
                video_event = {'video': video_urls[0]['lurl'], 'desc': playlist[0]['dsc']}
                successful_video_events.append(video_event)
    
        except KeyError as e:
            print(f"KeyError encountered for event ID {event_id}: {e}")
        except Exception as e:
            print(f"An error occurred for event ID {event_id}: {e}")

25.0% of 2021900513 complete
50.0% of 2021900513 complete
75.0% of 2021900513 complete
100.0% of 2021900513 complete
25.0% of 2021900509 complete
50.0% of 2021900509 complete
75.0% of 2021900509 complete
100.0% of 2021900509 complete
25.0% of 0021900883 complete
50.0% of 0021900883 complete
75.0% of 0021900883 complete
100.0% of 0021900883 complete
25.0% of 0021900879 complete
50.0% of 0021900879 complete
75.0% of 0021900879 complete
100.0% of 0021900879 complete
25.0% of 0021900884 complete
50.0% of 0021900884 complete
75.0% of 0021900884 complete
100.0% of 0021900884 complete


In [51]:
sve=successful_video_events

In [57]:
sve_backup=sve
print(len(sve),len(sve_backup))

940 940


In [59]:
print('before pruning:',len(sve))

make=[]
miss=[]

for i in sve:
    if not i['video']:
        sve.remove(i)
    else:
        if 'MISS' in i['desc']:
            miss.append(i)
        else:
            make.append(i)
print('after pruning:',len(sve),'\n makes:',len(make),'\n miss:',len(miss))

before pruning: 940
after pruning: 842 
 makes: 540 
 miss: 204


In [63]:
count=0
for m in make:
    if 'Shot' in make:
        count+=1
print(count)

0
